In [19]:
import lancedb
import pyarrow as pa
import pandas as pd
from datasets import load_dataset
from transformers import GPT2Tokenizer

Para este ejercicio descargamos una base de datos de mensajes de texto en inglés.

In [2]:
dataset = load_dataset("chirunder/text_messages")

In [3]:
df = pd.DataFrame(dataset['train'])
df.rename(columns={'text': 'texto'}, inplace=True)
df.head()

,texto
0,Top right I gained a little speed with the add...
1,They are heavier wheels though as are all the ...
2,Federally registering a trademark is more than...
3,I'll have to jog my memory from rooting a few ...
4,Unless you can afford to buy all new larger cl...


Para los siguientes ejercicios, voy a crear una variable del numero de palabras en cada mensaje de texto.

In [4]:
df['n'] = df['texto'].apply(lambda x: len(str(x).split()))
df = df[['n', 'texto']]
df = df.head(500)
df

,n,texto
0,13,Top right I gained a little speed with the add...
1,14,They are heavier wheels though as are all the ...
2,9,Federally registering a trademark is more than...
3,21,I'll have to jog my memory from rooting a few ...
4,10,Unless you can afford to buy all new larger cl...
...,...,...
495,4,Just thrilled for you!
496,12,I will be happy when the mornings are a little...
497,9,haha the irony of me having nice nuts LOOL.
498,11,"It is going the way of MySpace, Geocities and ..."


## Task 1:
A partir del dataframe df, crea df_tokenized (usando el Tokenizer de GPT2) con dos columnas pero con el texto tokenizado. Asegurate de que todos los embeddings tengan la misma longitud y los tokens sean enteros (todos enteros o todos doubles). 

In [20]:

# Cargar el tokenizer del modelo gpt2 de Huggingface
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')

# Supongamos que tu DataFrame original se llama df y tiene dos columnas: 'texto' y 'n'

# Tokenizar la columna de texto
df['vector'] = df['texto'].apply(lambda x: tokenizer(x)["input_ids"])

# Convertir la columna de enteros a strings
#df['n'] = df['n'].astype(str)

# Tokenizar la columna convertida a strings
#df['n_tokenized'] = df['n'].apply(lambda x: tokenizer(x)["input_ids"])

# Crear un nuevo DataFrame con las columnas tokenizadas
df_tokenized = df[['vector', 'n']]
df_tokenized['vector'] = df_tokenized['vector'].apply(lambda x: x[:300] + [0] * (300 - len(x)) if len(x) < 300 else x[:300])




# Mostrar el nuevo DataFrame tokenizado
print(df_tokenized)

/usr/local/lib/python3.12/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


                                                vector   n
0    [9126, 826, 314, 8618, 257, 1310, 2866, 351, 2...  13
1    [2990, 389, 20140, 13666, 996, 355, 389, 477, ...  14
2    [37, 5702, 453, 28336, 257, 16028, 318, 517, 6...   9
3    [40, 1183, 423, 284, 48342, 616, 4088, 422, 40...  21
4    [28042, 345, 460, 5368, 284, 2822, 477, 649, 4...  10
..                                                 ...  ..
495  [5703, 20536, 329, 345, 0, 0, 0, 0, 0, 0, 0, 0...   4
496  [40, 481, 307, 3772, 618, 262, 31143, 389, 257...  12
497  [71, 12236, 262, 21296, 286, 502, 1719, 3621, ...   9
498  [1026, 318, 1016, 262, 835, 286, 2011, 14106, ...  11
499  [40, 1101, 14442, 262, 3326, 12894, 2442, 319,...  23

[500 rows x 2 columns]


/tmp/ipykernel_2089/620747418.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_tokenized['vector'] = df_tokenized['vector'].apply(lambda x: x[:300] + [0] * (300 - len(x)) if len(x) < 300 else x[:300])


## Task 2:
Mete el dataframe a una tabla en una base de datos de LanceDB.

In [21]:
# Nos conectamos a una base de datos local
db = lancedb.connect("./.lancedb")
# Creamos una tabla en la base de datos
if "tabla" in db.table_names():
    db.drop_table("tabla")
db.create_table("tabla", df_tokenized)
db["tabla"].head()

pyarrow.Table
vector: fixed_size_list<item: float>[300]
  child 0, item: float
n: int64
----
vector: [[[9126,826,314,8618,257,...,0,0,0,0,0],[2990,389,20140,13666,996,...,0,0,0,0,0],[37,5702,453,28336,257,...,0,0,0,0,0],[40,1183,423,284,48342,...,0,0,0,0,0],[28042,345,460,5368,284,...,0,0,0,0,0]]]
n: [[13,14,9,21,10]]

## Task 3:
Haz una query estilo SQL a la tabla de la base de datos. Quiero que escribas la query equivalente y pongas la explicación de lo que está haciendo la consulta. Hint: usa la columna "n". 

In [22]:
# Resuelve el task 3 aqui
(db["tabla"].search()
    .where("n < 3")
    .select(["vector", "n"])
    .limit(10)
    .to_pandas())

,vector,n
0,"[5195.0, 41320.0, 2767.0, 30.0, 0.0, 0.0, 0.0,...",2
1,"[40732.0, 30.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",1
2,"[7376.0, 364.0, 8518.0, 13.0, 0.0, 0.0, 0.0, 0...",2
3,"[34.0, 5691.0, 13.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",1
4,"[10814.0, 15744.0, 260.0, 13.0, 0.0, 0.0, 0.0,...",2
5,"[38.0, 83.0, 6513.0, 612.0, 0.0, 0.0, 0.0, 0.0...",2
6,"[39507.0, 2087.0, 13.0, 0.0, 0.0, 0.0, 0.0, 0....",2
7,"[56.0, 18213.0, 45108.0, 0.0, 0.0, 0.0, 0.0, 0...",2
8,"[38052.0, 1342.0, 13.0, 0.0, 0.0, 0.0, 0.0, 0....",2
9,"[37.0, 415.0, 3477.0, 1705.0, 0.0, 0.0, 0.0, 0...",2


- Query en SQL equivalente:

SELECT texto, vector, n

FROM tabla

WHERE n < 3

LIMIT 10;
- Explicacion: Busca aquellos mensajes de la tabla que tengan menos de 3 palabras, regresa el texto, el texto tokenizado que es el vector y su longitud

## Task 4:
Inventa un mensaje de texto que tu podrías escribirle a un amigo. Tokenizalo y ponlo en el formato adecuado para hacer un vector query. Quiero que me regreses el mensaje más parecido al mensaje que inventaste (OJO: quiero el texto, no el embedding). HINT: Hay que decodear el resultado del query.

In [48]:
# Resuelve el task 4 aqui
mensaje = "I love you"
mensaje_tokenized = tokenizer(mensaje)["input_ids"]
print(mensaje_tokenized)

n = 300
def ajustar_vector(input, n):
    output = input[:n]
    
    # Si la lista es más corta que el tamaño objetivo, rellenar con 0.0
    while len(output) < n:
        output.append(0)
    
    return output
mensaje_tokenized = ajustar_vector(mensaje_tokenized, n)

print(mensaje_tokenized)


[40, 1842, 345]
[40, 1842, 345, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [49]:
mensajes = (db["tabla"].search(mensaje_tokenized)
    .metric("cosine") # Puede ser L2 o cosine
    .select(["vector", "n"])
    .limit(10)
    .to_pandas())

In [50]:
mensaje_mas_parecido = mensajes['vector'][0]

mensaje_decodificado = tokenizer.decode(mensaje_mas_parecido, skip_special_tokens=True)
print("El mensaje más parecido a I love you es: ", mensaje_decodificado)

mensaje_filtrado = [token for token in mensaje_mas_parecido if token != 0]
mensaje_decodificado = tokenizer.decode(mensaje_filtrado,skip_special_tokens=True)
print("Quitando los tokens 0: ", mensaje_decodificado)

El mensaje más parecido a I love you es:  I doubt they will.!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
Quitando los tokens 0:  I doubt they will.
